In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
from scipy import constants
import os
import glob
from matplotlib import rc
from sklearn.metrics import mean_squared_error

In [7]:
from InfluxData import InfluxData
client = InfluxData()
client.set_client('155.230.28.170',8086,'sslab','1231',database='kmaeq')
device_ids = client.get_device()

In [8]:
raw_data = {'dev_id':[],
            'Mean_PSD_x':[],
            'Mean_PSD_y':[],
            'Mean_PSD_z':[],
            'MSE_PSD_x':[],
            'MSE_PSD_y':[],
            'MSE_PSD_z':[]}
new_df = pd.DataFrame(raw_data)
new_df

,dev_id,Mean_PSD_x,Mean_PSD_y,Mean_PSD_z,MSE_PSD_x,MSE_PSD_y,MSE_PSD_z


In [9]:
EXPECT_DATASET_SIZE = 60000 # 10min data
def psd_processing(dev_id:str):
    cli = InfluxData()
    cli.set_client('155.230.28.170',8086,'sslab','1231',database='kmaeq')
    rs = cli.query(f"select * from acc_data where dev_id='{dev_id}' and time >= now() - 30m AND time <= now() - 20m")
    data = cli.resultSetToDF(rs)
    if type(data['acc_data']) is list: #  it means empty data
        print('empty data')
        return dev_id
    if len(data['acc_data']) < EXPECT_DATASET_SIZE*0.9: # not enough data size
        print('not enough data size')
        return dev_id
    data = data['acc_data'] # convert dict_dataframe to dataframe
    x = data['x']*constants.g
    y = data['y']*constants.g
    z = data['z']*constants.g
    offset_x = x - np.mean(x)
    offset_y = y - np.mean(y)
    offset_z = z - np.mean(z)
    Pxx, freqs_x = plt.psd(offset_x,NFFT=200,Fs=100)
    Pxx_true = [0 for i in range(len(Pxx))]
    x_mse = mean_squared_error(Pxx,Pxx_true)
    Pyy, freqs_y = plt.psd(offset_y,NFFT=200,Fs=100)
    Pyy_true = [0 for i in range(len(Pyy))]
    y_mse = mean_squared_error(Pyy,Pyy_true)
    Pzz, freqs_z = plt.psd(offset_z,NFFT=200,Fs=100)
    Pzz_true = [0 for i in range(len(Pzz))]
    z_mse = mean_squared_error(Pzz,Pzz_true)
    row = {'dev_id':dev_id,
            'Mean_PSD_x':np.mean(Pxx),
            'Mean_PSD_y':np.mean(Pyy),
            'Mean_PSD_z':np.mean(Pzz),
            'MSE_PSD_x': x_mse,
            'MSE_PSD_y': y_mse,
            'MSE_PSD_z': z_mse
            }
    return row

In [10]:
%%time
from concurrent.futures import ProcessPoolExecutor, as_completed, wait
pool = ProcessPoolExecutor(len(device_ids))
futures = []
for dev_id in device_ids:
    futures.append(pool.submit(psd_processing,dev_id))

err_devices = [] 

for x in as_completed(futures,timeout=120):
    if type(x.result()) is dict:
        new_df = new_df.append(x.result(),ignore_index=True)
    else:
        err_devices.append(x.result())

empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
not enough data size
empty data
empty data
empty data
not enough data size
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
not enough data size
empty data
not enough data size
empty data
not enough data size
empty data
empty data
empty data
empty data
empty data
not enough data size
empty data
not enough data size
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
empty data
not enough data size
empty data
empty data
empty data
empty data
empty data
not enough data size
not enough data size
not enough data size
not enough data size
not enough data size
not enough data size
not enough data size
not enough data size
not enough data size


Process Process-17:
Process Process-193:
Process Process-127:
Process Process-184:
Process Process-248:
Process Process-249:
Process Process-141:
Process Process-79:
Process Process-200:
Process Process-252:
Process Process-238:
Process Process-241:
Process Process-246:
Process Process-19:
Process Process-233:


cannot query to influxDB


Process Process-237:


cannot query to influxDB
cannot query to influxDB


...

  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/c

cannot query to influxDB


  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/concurrent/futures/process

failed qry contents:  select * from acc_data where dev_id='0206' and time >= now() - 30m AND time <= now() - 20m


  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/knusslab/anaconda3/envs/JupyterServer/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._targe

In [11]:
new_df

,dev_id,Mean_PSD_x,Mean_PSD_y,Mean_PSD_z,MSE_PSD_x,MSE_PSD_y,MSE_PSD_z
0,0064,2.298572e-03,3.309634e-03,0.000888,8.065001e-05,1.618696e-04,8.168634e-06
1,0057,2.085317e-06,2.001355e-06,0.000002,4.395754e-12,4.034671e-12,6.140002e-12
2,0085,2.515932e-06,2.328951e-06,0.000004,6.766213e-12,6.034993e-12,3.947526e-11
3,0027,5.510401e-08,2.460334e-06,0.000002,3.062933e-15,6.125980e-12,3.183304e-12
4,0073,4.192130e-04,2.160796e-06,0.000003,1.779505e-07,4.715288e-12,9.912683e-12
5,0026,1.877281e-06,2.204355e-06,0.000003,3.692302e-12,4.947467e-12,8.579022e-12
6,0062,1.968084e-06,1.412429e-06,0.000002,9.526784e-12,3.746721e-12,7.003432e-12
7,0079,1.857784e-06,3.778011e-06,0.000002,3.482019e-12,1.462275e-11,6.222581e-12
8,0015,1.869263e-06,9.433214e-08,0.000002,3.604871e-12,8.979279e-15,4.575525e-12
9,0040,1.725926e-06,6.014631e-07,0.000002,3.263209e-12,4.347852e-13,6.387868e-12


In [12]:
cant_qry = set(device_ids) - set(err_devices) - set(new_df['dev_id'])
cant_qry

{'0023',
 '0028',
 '0031',
 '0033',
 '0034',
 '0036',
 '0038',
 '0041',
 '0042',
 '0046',
 '0048',
 '0050',
 '0051',
 '0052',
 '0054',
 '0055',
 '0056',
 '0060',
 '0065',
 '0066',
 '0069',
 '0070',
 '0071',
 '0075',
 '0076',
 '0080',
 '0081',
 '0082',
 '0083',
 '0086',
 '0089',
 '0091',
 '0092',
 '0093',
 '0098',
 '0100',
 '0101',
 '0102',
 '0104',
 '0105',
 '0106',
 '0107',
 '0108',
 '0110',
 '0111',
 '0112',
 '0113',
 '0114',
 '0115',
 '0119',
 '0120',
 '0122',
 '0123',
 '0124',
 '0125',
 '0127',
 '0128',
 '0134',
 '0135',
 '0136',
 '0137',
 '0138',
 '0139',
 '0140',
 '0141',
 '0143',
 '0144',
 '0145',
 '0146',
 '0149',
 '0161',
 '0162',
 '0163',
 '0164',
 '0165',
 '0167',
 '0170',
 '0171',
 '0172',
 '0173',
 '0175',
 '0176',
 '0177',
 '0178',
 '0179',
 '0180',
 '0181',
 '0182',
 '0183',
 '0184',
 '0185',
 '0186',
 '0187',
 '0188',
 '0189',
 '0190',
 '0192',
 '0194',
 '0198',
 '0203',
 '0205',
 '0206',
 '0208',
 '0209',
 '0210',
 '0212',
 '0213',
 '0214',
 '0215',
 '0216',
 '0217',
 

In [13]:
dev_id = new_df['dev_id']
Feature = new_df.loc[:,new_df.columns != 'dev_id']

In [14]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(Feature)
Feature = ss.transform(Feature)
Feature

array([[ 6.06420010e+00,  6.16439551e+00,  6.16111905e+00,
         6.16439901e+00,  6.16441400e+00,  6.16441338e+00],
       [-1.88820039e-01, -1.62039319e-01, -1.71293995e-01,
        -1.62588754e-01, -1.62222112e-01, -1.62344456e-01],
       [-1.87647534e-01, -1.61412735e-01, -1.58302836e-01,
        -1.62588568e-01, -1.62222034e-01, -1.62318637e-01],
       [-1.94348033e-01, -1.61161440e-01, -1.76445559e-01,
        -1.62589098e-01, -1.62222030e-01, -1.62346746e-01],
       [ 9.46961789e-01, -1.61734360e-01, -1.66800672e-01,
        -1.48628890e-01, -1.62222085e-01, -1.62341534e-01],
       [-1.89386495e-01, -1.61651046e-01, -1.71018166e-01,
        -1.62588809e-01, -1.62222076e-01, -1.62342567e-01],
       [-1.89139250e-01, -1.63165745e-01, -1.73972697e-01,
        -1.62588351e-01, -1.62222123e-01, -1.62343787e-01],
       [-1.89439582e-01, -1.58641150e-01, -1.71368094e-01,
        -1.62588825e-01, -1.62221698e-01, -1.62344392e-01],
       [-1.89408327e-01, -1.65686840e-01, -1.737

In [ ]:
k_means_centers = np.load('PSD_k-means_center.npy')

In [ ]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=6)
k_means.cluster_centers_ = k_means_centers
k_means_labels = k_means.predict(Feature)
k_means_result = pd.concat([])